In [37]:
import pandas as pd
import math
import numpy as np
from os import path
from config import ROOT_PATH, DATA_PATH

# Theory to calculate the abundance and fishing mortality matrix

## 1) Calculate diagonal abundance and fishing mortality matrices 

The first step will calculate the diagonal abundance and fishing mortality matrices, assuming a terminal fishing moratlity $ F_{3} $ and fix natural mortality M

1. Calculation of abundance for oldest age: $ \large (1) N_{8} = \frac{C_{8} * Z_{8}}{F_{8} * \ e^{(1 - Z_{8})}} $
2. Once $ N_{8} $ is known, we calculate the fishing mortality of same cohort the year earlier $ F_{7} $ so that $ N_{8} $ was left. To do so we use the catch equation at age: $ \large C_{7} = \frac{F_{7}}{Z_{7}} * e^{(1 - Z_{7})} $
3. Last we calculate $ N_{7} $ using teh exponential decay function for $ N_{t} $: $ \large N_{7} =  N_{8} * e^{(Z_{7})} = N_{8} * e^{(F_{7} + M)} $
4. We repeat the steps 2 and 3 until we calculate $ N_{1} $

## 2) Recalculate the abundances for a submatrix that excludes last year and last age

The abundance matrix is modified by recalculating the submatrix that excludes the last row (last year) and column (last age). The formula to recalculate it is: $ \large N_{t} =  N_{t+1}e^{M} + C_{t}e^{\frac{M}{2}} $. This process will generate a new abundance matrix.

## 3) Recalculate the fishing mortality matrix

1. Calculate first the submatrix that excludes the last column (last age) with the formula: $ \large log_n (\frac{N_{t,a}}{N_{t+1,a+1}}) - M$
2. Calculate the last column (last age fishing mortality) as the average of the fishing mortality of the previoius ages.  In our cases we would get use only from age 3 onwards as the fishing mortality of those ages is very small and the natural one very large.


In [30]:
# Read catch, and mortality rates (1)
# ---> read natural mortality matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'm_mortality_rate.csv')
m = pd.read_csv(data_path)

# ---> read fishing mortality matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'f_mortality_rate.csv')
f = pd.read_csv(data_path)

# --> read catch matrix
data_path =path.join(DATA_PATH, 'project', 'vpa', 'catch_rate.csv')
c = pd.read_csv(data_path)

# --> abundance matrix
n = f.copy()
n[n.columns[1:]] = 0


## 1.1) Calculate last column and row of N matrix of abundance matrix

Use catch equation $ \large N_{t} = \frac{C_{t} * Z_{t}}{F_{t} * \ e^{(1 - Z_{t})}} $


In [117]:
# Calculate abundance for last column (age 8 for all years)
ct = c.iloc[:, -1]
zt = f.iloc[:, -1] + m.iloc[:, -1]
ft = f.iloc[:, -1]
exp_z = np.exp(pd.Series([1] * len(zt)) - zt)
# update last column
n.iloc[:, -1] = round(ct * zt / ft * exp_z, 2)

In [118]:
# Calculate abundance for last row (all ages for the last year)
ct = c.iloc[-1, 1:].reset_index(drop=True)
zt = (f.iloc[-1, 1:] + m.iloc[-1, 1:]).reset_index(drop=True)
ft = f.iloc[-1, 1:].reset_index(drop=True)
exp_z = np.exp(pd.Series([1] * len(zt)) - zt)
# update last rowex
n.iloc[-1, 1:] = round(ctr * ztr / ftr * exp_z, 2)

## 1.2) Calculate rest of abundance matrix using Pope's equation

The Pope's equation $ \large N_{t, a} =  N_{t+1, a+1}e^{M} + C_{t}e^{\frac{M}{2}} $ is used to calculate the rest of the rows

In [128]:
for col in range(len(n.columns)-1, 0, -1):
    nt1 = n.iloc[:-1, col]
    e_m = np.exp(m.iloc[:-1, col])
    ct = c.iloc[:-1, col]
    em_2 = np.exp(m.iloc[:-1, col] / 2)
    n.iloc[:-1, col] = (nt1 * e_m) + (ct * em_2)